In [11]:
import requests
import json
import time
from string import ascii_uppercase
from pathlib import Path
import pandas as pd

BATCH_SIZE = 1000
MAX_PAGES_PER_COMBO = 10
SLEEP_TIME = 0.5

# Output base folder
base_dir = Path("raw/hsc_achievers_yearly")
base_dir.mkdir(parents=True, exist_ok=True)

# Process each year from 2018 to 2024
for year in range(2018, 2019):
    print(f"\n📅 Starting year: {year}")
    index_name = f"prod_nesa_{year}_hsc_distinguished_achievers_rectified"
    BASE_URL = f"https://www.nsw.gov.au/api/v1/elasticsearch/{index_name}/_search"

    year_dir = base_dir / str(year)
    year_dir.mkdir(parents=True, exist_ok=True)

    seen_ids = set()

    # Track all source entries to export as CSV later
    full_year_data = []

    for first_initial in ascii_uppercase:
        for last_initial in ascii_uppercase:
            pair_id = f"{first_initial}_{last_initial}"
            pair_path = year_dir / f"{pair_id}.jsonl"

            if pair_path.exists():
                print(f"✅ Skipping {pair_id} for {year} (already saved)")
                continue

            print(f"🔍 {year} - {pair_id}")

            for page in range(MAX_PAGES_PER_COMBO):
                offset = page * BATCH_SIZE

                query = {
                    "from": offset,
                    "size": BATCH_SIZE,
                    "query": {
                        "bool": {
                            "must": [
                                {
                                    "query_string": {
                                        "query": f"{first_initial}*",
                                        "fields": ["first_name"],
                                        "rewrite": "scoring_boolean"
                                    }
                                },
                                {
                                    "query_string": {
                                        "query": f"{last_initial}*",
                                        "fields": ["last_name"],
                                        "rewrite": "scoring_boolean"
                                    }
                                }
                            ]
                        }
                    },
                    "sort": [{"_score": {"order": "desc"}}]
                }

                params = {
                    "source_content_type": "application/json",
                    "source": json.dumps(query)
                }

                try:
                    response = requests.get(BASE_URL, params=params)
                    response.raise_for_status()
                    data = response.json()
                    hits = data.get("hits", {}).get("hits", [])

                    if not hits:
                        break

                    new_hits = []
                    for hit in hits:
                        if hit["_id"] not in seen_ids:
                            seen_ids.add(hit["_id"])
                            new_hits.append(hit)
                            full_year_data.append(hit["_source"])

                    if new_hits:
                        with pair_path.open("a") as f:
                            for hit in new_hits:
                                f.write(json.dumps(hit) + "\n")

                        print(f"📦 Page {page+1}: {len(new_hits)} new records")
                    else:
                        print(f"🛑 Page {page+1}: No new records")
                        break

                    time.sleep(SLEEP_TIME)

                except Exception as e:
                    print(f"❌ Error: {e}")
                    break

    # Save all results for the year as CSV
    if full_year_data:
        df = pd.DataFrame(full_year_data)
        csv_path = year_dir.parent / f"{year}_distinguished_achievers.csv"
        df.to_csv(csv_path, index=False)
        print(f"💾 Saved full CSV for {year} with {len(df)} records")

print("\n✅ All years completed.")


📅 Starting year: 2018
🔍 2018 - A_A
📦 Page 1: 292 new records
🔍 2018 - A_B
📦 Page 1: 323 new records
🔍 2018 - A_C
📦 Page 1: 374 new records
🔍 2018 - A_D
📦 Page 1: 184 new records
🔍 2018 - A_E
📦 Page 1: 59 new records
🔍 2018 - A_F
📦 Page 1: 166 new records
🔍 2018 - A_G
📦 Page 1: 225 new records
🔍 2018 - A_H
📦 Page 1: 311 new records
🔍 2018 - A_I
📦 Page 1: 28 new records
🔍 2018 - A_J
📦 Page 1: 119 new records
🔍 2018 - A_K
📦 Page 1: 261 new records
🔍 2018 - A_L
📦 Page 1: 349 new records
🔍 2018 - A_M
📦 Page 1: 463 new records
🔍 2018 - A_N
📦 Page 1: 168 new records
🔍 2018 - A_O
📦 Page 1: 79 new records
🔍 2018 - A_P
📦 Page 1: 211 new records
🔍 2018 - A_Q
📦 Page 1: 16 new records
🔍 2018 - A_R
📦 Page 1: 145 new records
🔍 2018 - A_S
📦 Page 1: 434 new records
🔍 2018 - A_T
📦 Page 1: 252 new records
🔍 2018 - A_U
📦 Page 1: 6 new records
🔍 2018 - A_V
📦 Page 1: 69 new records
🔍 2018 - A_W
📦 Page 1: 222 new records
🔍 2018 - A_X
📦 Page 1: 29 new records
🔍 2018 - A_Y
📦 Page 1: 82 new records
🔍 2018 - A_

In [9]:
for year in range(2017, 2018):
    print(year)

2017


In [2]:
import pandas as pd
import json
from pathlib import Path

# Folder containing all the pair files
jsonl_dir = Path("raw/hsc_achievers")
output_csv_path = Path("raw/hsc_achievers_combined.csv")

# Load all jsonl files
records = []
for file in jsonl_dir.glob("*.jsonl"):
    with open(file, "r") as f:
        for line in f:
            try:
                record = json.loads(line.strip())
                source = record.get("_source", {})
                records.append(source)
            except json.JSONDecodeError:
                print(f"❌ Could not parse line in {file}")
                continue

# Convert to DataFrame and save as CSV
df = pd.DataFrame(records)
df.to_csv(output_csv_path, index=False)
print(f"✅ Combined {len(df)} records into {output_csv_path}")

✅ Combined 41945 records into raw/hsc_achievers_combined.csv
